In [1]:
!pip install pykrige

In [2]:
!pip install geopandas

In [3]:
!pip install contextily

In [4]:
!pip install lightgbm

In [5]:
import numpy as np
import pandas as pd
import pyproj
from sklearn.model_selection import train_test_split
import geopandas as gpd
from shapely.geometry import Point
from scipy.optimize import minimize
import contextily as ctx
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.impute import SimpleImputer
from sklearn.model_selection import GridSearchCV
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

C:\Users\ghala\anaconda3\lib\site-packages\dask\dataframe\__init__.py:31: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [6]:
#Step 2: Load the Dataset
dataset = pd.read_csv('modified_dataset.csv')
dataset.head()

,STATION_ID,OBSERVATION_DATE,LATITUDE,LONGITUDE,ELEVATION,AIR_TEMPERATURE,AIR_TEMPERATURE_QUALITY,AIR_TEMPERATURE_DEW_POINT,AIR_TEMPERATURE_DEW_POINT_QUALITY,ATMOSPHERIC_SEA_LEVEL_PRESSURE,ATMOSPHERIC_SEA_LEVEL_PRESSURE_QUALITY
0,41112099999,2000-01-01 01:00:00,18.240367,42.656625,2090.31,6.0,Passed all quality control checks,-1.0,Passed all quality control checks,9999.9,Passed gross limits check if element is present
1,41112099999,2000-01-01 02:00:00,18.240367,42.656625,2090.31,6.0,Passed all quality control checks,-1.0,Passed all quality control checks,9999.9,Passed gross limits check if element is present
2,41112099999,2000-01-01 03:00:00,18.240367,42.656625,2090.31,6.0,Passed all quality control checks,-1.0,Passed all quality control checks,9999.9,Passed gross limits check if element is present
3,41112099999,2000-01-01 04:00:00,18.240367,42.656625,2090.31,7.0,Passed all quality control checks,-3.0,Passed all quality control checks,9999.9,Passed gross limits check if element is present
4,41112099999,2000-01-01 05:00:00,18.240367,42.656625,2090.31,9.0,Passed all quality control checks,-3.0,Passed all quality control checks,9999.9,Passed gross limits check if element is present


In [7]:
print("Before feature Engineering:")
dataset.shape

Before feature Engineering:


(1000, 11)

In [8]:
dataset['OBSERVATION_DATE'] = pd.to_datetime(dataset['OBSERVATION_DATE'])

In [9]:
dataset['DAY_OF_YEAR'] = dataset['OBSERVATION_DATE'].dt.dayofyear
dataset['HOUR'] = dataset['OBSERVATION_DATE'].dt.hour

In [10]:
geodesic = pyproj.Geod(ellps='WGS84')

def get_dist_bearing(p1, p2):
    ''''
    p1: lat,lon
    p2: lat,lon
    in degrees
    '''
    lat1, lon1 = p1
    lat2, lon2 = p2
    fwd_azimuth, back_azimuth, distance = geodesic.inv(lon1, lat1, lon2, lat2)
    return distance, fwd_azimuth

In [11]:
# Extract coordinates from the dataset and convert to a set of tuples
coords = list(zip(dataset['LATITUDE'], dataset['LONGITUDE']))

In [12]:
def geometric_median(points : list):
    points = np.array(points)  # Convert to numpy array for array operations
    # Initial guess for the geometric median (centroid)
    initial_guess = np.mean(points, axis=0)

    # Define the objective function (sum of distances to minimize)
    def objective_function(center):
        return np.sum(np.linalg.norm(points - center, axis=1))

    # Use numerical optimization (minimization) to find the geometric median
    result = minimize(objective_function, initial_guess, method='Nelder-Mead')

    return result.x

center = geometric_median(coords)

In [13]:
# Calculate distance and bearing for each coordinate relative to the center
dataset['dist_bearing'] = dataset.apply(lambda row: get_dist_bearing(center, (row['LATITUDE'], row['LONGITUDE'])), axis=1)

In [14]:
dataset['DIST'], dataset['DIREC'] = zip(*dataset['dist_bearing'])

In [15]:
features = [
    'DIST',  # Geographic coordinate: Latitude
    'DIREC',  # Geographic coordinate: Longitude
    'ELEVATION',  # Elevation above sea level
    'DAY_OF_YEAR',  # Day of the year (from OBSERVATION_DATE)
    'HOUR',  # Hour of the day (from OBSERVATION_DATE)
    'AIR_TEMPERATURE_DEW_POINT'  # Dew point temperature
]

In [16]:
print("After feature Engineering:")
dataset.shape

After feature Engineering:


(1000, 16)

In [17]:
#Step 6: Data Cleaning
# Remove duplicate rows
dataset = dataset.drop_duplicates()

In [18]:
#Impute Missing Values

# Create an imputer object and fit it to the dataset
imputer = SimpleImputer(strategy='mean')
dataset_imputed = imputer.fit_transform(dataset[features])

#dataset[features].isnull().sum()

In [19]:
# Step 8: Statistics Summary
print("Dataset Statistics:")
dataset.describe()

Dataset Statistics:


,STATION_ID,OBSERVATION_DATE,LATITUDE,LONGITUDE,ELEVATION,AIR_TEMPERATURE,AIR_TEMPERATURE_DEW_POINT,ATMOSPHERIC_SEA_LEVEL_PRESSURE,DAY_OF_YEAR,HOUR,DIST,DIREC
count,1.000000e+03,1000,1.000000e+03,1.000000e+03,1.000000e+03,1000.000000,1000.000000,1.000000e+03,1000.000000,1000.000000,1.000000e+03,1.000000e+03
mean,4.111210e+10,2000-01-22 15:57:17.760000,1.824037e+01,4.265662e+01,2.090310e+03,17.271000,9.746400,9.999900e+03,22.184000,11.538000,5.452022e-08,-9.706607e+01
min,4.111210e+10,2000-01-01 01:00:00,1.824037e+01,4.265662e+01,2.090310e+03,5.000000,-12.000000,9.999900e+03,1.000000,0.000000,5.452022e-08,-9.706607e+01
25%,4.111210e+10,2000-01-11 20:45:00,1.824037e+01,4.265662e+01,2.090310e+03,11.000000,3.000000,9.999900e+03,11.000000,6.000000,5.452022e-08,-9.706607e+01
50%,4.111210e+10,2000-01-22 19:30:00,1.824037e+01,4.265662e+01,2.090310e+03,14.000000,7.000000,9.999900e+03,22.000000,12.000000,5.452022e-08,-9.706607e+01
75%,4.111210e+10,2000-02-02 11:15:00,1.824037e+01,4.265662e+01,2.090310e+03,18.000000,10.000000,9.999900e+03,33.000000,17.000000,5.452022e-08,-9.706607e+01
max,4.111210e+10,2000-02-13 06:00:00,1.824037e+01,4.265662e+01,2.090310e+03,999.900000,999.900000,9.999900e+03,44.000000,23.000000,5.452022e-08,-9.706607e+01
std,0.000000e+00,NaN,6.042635e-14,5.118468e-13,4.140272e-11,54.083493,62.990863,1.892696e-10,12.488238,6.868751,9.798719e-22,4.834108e-13


In [20]:
# Extract relevant features and target variable
X = dataset[['LONGITUDE', 'LATITUDE', 'ELEVATION', 'DAY_OF_YEAR', 'HOUR']].values
y = dataset['AIR_TEMPERATURE'].values

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [22]:
# LightGBM
# Define parameter grid
param_grid_lgbm = {
    'num_leaves': [15, 31, 63],
    'learning_rate': [0.01, 0.05, 0.1],
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7]
}

In [23]:
# Create untrained LightGBM model
lgbm_model = lgb.LGBMRegressor()

In [24]:
# Create GridSearchCV object with the untrained model
grid_lgbm = GridSearchCV(lgbm_model, param_grid_lgbm, cv=5, scoring='neg_mean_squared_error')

In [25]:
# Fit the grid search
grid_lgbm.fit(X_train, y_train)

C:\Users\ghala\anaconda3\lib\site-packages\joblib\externals\loky\backend\context.py:150: UserWarning: Could not find the number of physical cores for the following reason:
found 0 physical cores < 1
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "C:\Users\ghala\anaconda3\lib\site-packages\joblib\externals\loky\backend\context.py", line 245, in _count_physical_cores
    raise ValueError(


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000158 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 69
[LightGBM] [Info] Number of data points in the train set: 640, number of used features: 2
[LightGBM] [Info] Start training from score 18.923281
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, bes

[LightGBM] [Info] Total Bins 69
[LightGBM] [Info] Number of data points in the train set: 640, number of used features: 2
[LightGBM] [Info] Start training from score 19.056719
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf

[LightGBM] [Info] Total Bins 69
[LightGBM] [Info] Number of data points in the train set: 640, number of used features: 2
[LightGBM] [Info] Start training from score 19.056719
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

...

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Info] Total Bins 69
[LightGBM] [Info] Number of data points in the train set: 800, number of used features: 2
[LightGBM] [Info] Start training from score 18.065625


GridSearchCV(cv=5, estimator=LGBMRegressor(),
             param_grid={'learning_rate': [0.01, 0.05, 0.1],
                         'max_depth': [3, 5, 7], 'n_estimators': [50, 100, 200],
                         'num_leaves': [15, 31, 63]},
             scoring='neg_mean_squared_error')

In [26]:
import joblib
joblib.dump(grid_lgbm,'lgbm_model.pkl')

['lgbm_model.pkl']

In [27]:
# Make predictions on the test data using the best estimator found by grid search
y_pred_lgbm_tuned = grid_lgbm.best_estimator_.predict(X_test)
# Make predictions on the training data using the best estimator found by grid search
y_train_pred_lgbm_tuned = grid_lgbm.best_estimator_.predict(X_train)

In [28]:
#Print best parameters and score
print("Best parameters for LightGBM:", grid_lgbm.best_params_)
print("Best score for LightGBM:", -grid_lgbm.best_score_)

Best parameters for LightGBM: {'learning_rate': 0.01, 'max_depth': 7, 'n_estimators': 50, 'num_leaves': 15}
Best score for LightGBM: 3665.6533710699923


In [29]:
# Evaluate the performance of the tuned LightGBM model on both training and test sets
print("Performance metrics for LightGBM after tuning:")
print("Training Set:")

# Evaluate Mean Squared Error (MSE) on training set
mse_lgbm_tuned_train = mean_squared_error(y_train, y_train_pred_lgbm_tuned)
print("Mean Squared Error (MSE):", mse_lgbm_tuned_train)
# Evaluate Mean Absolute Error (MAE) on training set
mae_lgbm_tuned_train = mean_absolute_error(y_train, y_train_pred_lgbm_tuned)
print("Mean Absolute Error (MAE):", mae_lgbm_tuned_train)
# Evaluate Root Mean Squared Error (RMSE) on training set
rmse_lgbm_tuned_train = np.sqrt(mse_lgbm_tuned_train)
print("Root Mean Squared Error (RMSE):", rmse_lgbm_tuned_train)
# Evaluate R-squared (R2) Score on training set
r2_lgbm_tuned_train = r2_score(y_train, y_train_pred_lgbm_tuned)
print("R-squared (R2) Score:", r2_lgbm_tuned_train)

print("\nTest Set:")

# Evaluate Mean Squared Error (MSE) on test set
mse_lgbm_tuned = mean_squared_error(y_test, y_pred_lgbm_tuned)
print("Mean Squared Error (MSE):", mse_lgbm_tuned)
# Evaluate Mean Absolute Error (MAE) on test set
mae_lgbm_tuned = mean_absolute_error(y_test, y_pred_lgbm_tuned)
print("Mean Absolute Error (MAE):", mae_lgbm_tuned)
# Evaluate Root Mean Squared Error (RMSE) on test set
rmse_lgbm_tuned = np.sqrt(mse_lgbm_tuned)
print("Root Mean Squared Error (RMSE):", rmse_lgbm_tuned)
# Evaluate R-squared (R2) Score on test set
r2_lgbm_tuned = r2_score(y_test, y_pred_lgbm_tuned)
print("R-squared (R2) Score:", r2_lgbm_tuned)

Performance metrics for LightGBM after tuning:
Training Set:
Mean Squared Error (MSE): 3549.8124124436827
Mean Absolute Error (MAE): 7.854028164967308
Root Mean Squared Error (RMSE): 59.580302218465484
R-squared (R2) Score: 0.02629702516762511

Test Set:
Mean Squared Error (MSE): 42.288938449952326
Mean Absolute Error (MAE): 4.610698186235653
Root Mean Squared Error (RMSE): 6.502994575574573
R-squared (R2) Score: -1.7940243739952209


In [30]:
pip install streamlit

Note: you may need to restart the kernel to use updated packages.


In [31]:
pip install altair==4.2.2

Note: you may need to restart the kernel to use updated packages.


In [32]:
pip install --upgrade pandas dask lightgbm

Note: you may need to restart the kernel to use updated packages.


In [33]:
%%writefile lgbm_model.py
# ----------------------
import streamlit as st
import joblib

import numpy as np
import pandas as pd
import pyproj
from sklearn.model_selection import train_test_split
import geopandas as gpd
from shapely.geometry import Point
from scipy.optimize import minimize
import contextily as ctx
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import Ridge
import lightgbm as lgb
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import r2_score
import altair as alt
from datetime import datetime, timedelta

# ----------------------
def geometric_median(points: list):
    points = np.array(points)
    initial_guess = np.mean(points, axis=0)
    def objective_function(center):
        return np.sum(np.linalg.norm(points - center, axis=1))
    result = minimize(objective_function, initial_guess, method='Nelder-Mead')
    return result.x

def load_data(file_path):
    dataset = pd.read_csv(file_path)
    return dataset

def feature_engineering(dataset):
    dataset['OBSERVATION_DATE'] = pd.to_datetime(dataset['OBSERVATION_DATE'])
    dataset['DAY_OF_YEAR'] = dataset['OBSERVATION_DATE'].dt.dayofyear
    dataset['HOUR'] = dataset['OBSERVATION_DATE'].dt.hour
    coords = list(zip(dataset['LATITUDE'], dataset['LONGITUDE']))
    center = geometric_median(coords)
    dataset['dist_bearing'] = dataset.apply(lambda row: get_dist_bearing(center, (row['LATITUDE'], row['LONGITUDE'])), axis=1)
    dataset['DIST'], dataset['DIREC'] = zip(*dataset['dist_bearing'])
    return dataset

def data_cleaning(dataset):
    dataset = dataset.drop_duplicates()
    return dataset

def get_dist_bearing(center, point):
    geodisc = pyproj.Geod(ellps='WGS84')
    lon1, lat1 = center
    lon2, lat2 = point
    fwd_azimuth, back_azimuth, distance = geodisc.inv(lon1, lat1, lon2, lat2)
    return distance, fwd_azimuth

def main():
    st.title("Geospatial Analysis App!")
    st.sidebar.title("Menu")
    menu = st.sidebar.selectbox("Select an option", ["Overview", "Data Analysis", "Temperature Prediction"])

    if menu == "Overview":
        st.subheader("Overview")
        st.write("Welcome to the Geospatial Analysis App!")
        st.write("Geospatial analysis is a powerful approach for examining and interpreting data that possesses a spatial or geographical component. By leveraging geographic information systems (GIS) and various analytical techniques, geospatial analysis allows us to uncover valuable insights about our world.")
        st.write("In this project, our focus is on exploring climate data within the context of the Kingdom of Saudi Arabia. By harnessing geospatial analysis techniques, we aim to gain a deeper understanding of the intricate relationships between climatic factors and geographical features within this region.")
        st.write("The Kingdom of Saudi Arabia, with its vast and diverse landscape, presents an intriguing dataset for geospatial analysis. Across different geographical locations offers a rich tapestry for exploration.")
        st.write("Through this app, we'll delve into climate data sourced from various observation stations across Saudi Arabia. By examining parameters such as temperature, elevation, and humidity, we'll unravel spatial patterns and correlations, shedding light on the complex interplay between environmental factors.")
        st.write("Join us on this journey as we employ geospatial analysis techniques to decipher the climate data of the Kingdom of Saudi Arabia, uncovering insights that may help scientific research.")

    elif menu == "Data Analysis":
        st.subheader("Data Analysis Section")
        st.write("Welcome to the Data Analysis section! In this section, we will explore various geospatial analysis techniques to gain insights into climate data related to the Kingdom of Saudi Arabia.")
        st.write("Geospatial analysis involves analyzing and visualizing data that has a geographical or spatial component. It helps us understand how different climatic parameters vary across space and time.")
        st.write("In this project, we are particularly interested in studying climate data, including temperature, elevation, and humidity, to uncover spatial patterns and correlations in the Kingdom of Saudi Arabia.")
        st.write("One of the key techniques we'll be using is kriging, a geostatistical method for interpolating spatial data. However, since kriging is not directly supported by Streamlit's mapping functionality, we'll focus on other regression models for analysis.")

        file_path = 'modified_dataset.csv'
        dataset = load_data(file_path)
        dataset = feature_engineering(dataset)
        dataset = data_cleaning(dataset)

        features = [
            'DIST',
            'DIREC',
            'ELEVATION',
            'DAY_OF_YEAR',
            'HOUR',
            'AIR_TEMPERATURE_DEW_POINT']

        st.write("Let's start by exploring regression models for predicting air temperature based on various features.")
        st.write("We'll evaluate the following regression models:")
        st.write("And after training and testing each model, we'll compare their performance using R-squared (R2).")
        models = {
            'KNN': KNeighborsRegressor(),
            'LightGBM': lgb.LGBMRegressor(),
            'Ridge Regression': Ridge(),
            'Gradient Boosting Regressor': GradientBoostingRegressor()}

        st.write("Here are our models ranked based on the result after training:")
        for model_name in models:
            st.write(f"- {model_name}")
            

        st.write("In conclusion, while KNN and LightGBM emerged as the top-performing models, and they are the most appropriate models for predicting air temperature based on the given features")

    elif menu == "Temperature Prediction":
        st.subheader("Temperature Prediction with Categorization")

        #Load the LightGBM model, the reason that we choose LightGBM because after tuning, there is improvement in R-squared on the training set,and it's compatible with both small and large datasets.
        model = joblib.load('lgbm_model.pkl')

        st.write("Enter numeric values to predict the temperature:")
        
        # User input - input_values
        start_date = datetime.today() - timedelta(days=365*24)
        end_date = datetime.today()
        
        Longitude_values = st.text_input('Enter Longitude:', value='0.0')
        Latitude_values = st.text_input('Enter Latitude:', value='0.0')
        Elevation_values = st.text_input('Enter Elevation(Integer):', value='0')
        Day_Of_Year_values = st.date_input("Choose the Day of the Year:",value=end_date, min_value=start_date, max_value=end_date)
        Hour_values = st.selectbox("Choose the Hour :", [str(hour) for hour in range(1, 25)])
        day_of_year = Day_Of_Year_values.timetuple().tm_yday
        
        # Categorization function
        def categorize_temperature(temp):
            if temp < 12:
                return "Cold"
            elif 12 <= temp < 25:
                return "Normal"
            else:
                return "Hot"

        # Prediction + categorization
        if st.button('Predict Temperature'):
          
            try:
                all_inputs = np.array([[float(Longitude_values), float(Latitude_values), int(Elevation_values), day_of_year, float(Hour_values)]], dtype=np.float32)                #input_list = np.array([[float(x) for x in input_values.split(',')]])
                prediction = model.predict(all_inputs)[0]
                category = categorize_temperature(prediction)
                st.write(f"Predicted Air Temperature (LightGBM): {prediction:.2f} °C")
                st.write(f"Temperature Category: {category}")
            except ValueError:
                st.write("Invalid input. Please enter numeric values")

if __name__ == "__main__":
    main()

Overwriting lgbm_model.py
